In [1]:
import os
os.environ["OPENAI_API_KEY"] = ""
OPENAI_API_KEY = ""

## LangChain

In [2]:
!pip install --upgrade httpx==0.28.1
!pip install langchain
!pip install langchain openai
!pip install langchain_community
!pip install langchain_core
!pip install langchain_experimental
!pip install -U duckduckgo-search
!pip install deep-translator
!pip install openai
!pip install wikipedia
!pip install pandas numpy
!pip install nltk

In [3]:
# Import from LangChain
import os
from langchain.tools import Tool, DuckDuckGoSearchResults
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from deep_translator import GoogleTranslator
from langchain.memory import ConversationBufferMemory
import wikipedia
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Tuple, Optional
import re
import nltk

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


### Pandas Analysis Tool Logic

In [4]:
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)


# Generate sample data
n_rows = 1000

# Generate dates
start_date = datetime(2022, 1, 1)
dates = [start_date + timedelta(days=i) for i in range(n_rows)]

# Define data categories
makes = ['Toyota', 'Honda', 'Ford', 'Chevrolet', 'Nissan', 'BMW', 'Mercedes', 'Audi', 'Hyundai', 'Kia']
models = ['Sedan', 'SUV', 'Truck', 'Hatchback', 'Coupe', 'Van']
colors = ['Red', 'Blue', 'Black', 'White', 'Silver', 'Gray', 'Green']

# Create the dataset
data = {
    'Date': dates,
    'Make': np.random.choice(makes, n_rows),
    'Model': np.random.choice(models, n_rows),
    'Color': np.random.choice(colors, n_rows),
    'Year': np.random.randint(2015, 2023, n_rows),
    'Price': np.random.uniform(20000, 80000, n_rows).round(2),
    'Mileage': np.random.uniform(0, 100000, n_rows).round(0),
    'EngineSize': np.random.choice([1.6, 2.0, 2.5, 3.0, 3.5, 4.0], n_rows),
    'FuelEfficiency': np.random.uniform(20, 40, n_rows).round(1),
    'SalesPerson': np.random.choice(['Alice', 'Bob', 'Charlie', 'David', 'Eva'], n_rows)
}

# Create DataFrame and sort by date
df = pd.DataFrame(data).sort_values('Date')

# Display sample data and statistics
print("\nFirst few rows of the generated data:")
print(df.head())

print("\nDataFrame info:")
df.info()

print("\nSummary statistics:")
print(df.describe())


First few rows of the generated data:
        Date     Make      Model   Color  Year     Price  Mileage  EngineSize  \
0 2022-01-01  Hyundai        SUV    Gray  2022  23801.91  16985.0         2.5   
1 2022-01-02   Toyota      Coupe     Red  2015  62512.64  85391.0         2.5   
2 2022-01-03      BMW  Hatchback  Silver  2019  20858.55  73751.0         3.5   
3 2022-01-04   Toyota        Van     Red  2016  43102.45  35477.0         4.0   
4 2022-01-05  Hyundai        Van  Silver  2022  54166.50  58744.0         3.5   

   FuelEfficiency SalesPerson  
0            36.5         Eva  
1            23.7         Eva  
2            38.9       Alice  
3            29.2     Charlie  
4            22.7     Charlie  

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            1000 non-null   datetime64[ns]
 1 

In [5]:
pandas_agent = create_pandas_dataframe_agent(
    ChatOpenAI(model="gpt-4", temperature=0, openai_api_key= OPENAI_API_KEY),
    df,
    verbose=True,
    allow_dangerous_code=True,  # Only set this if you're comfortable with code execution
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

<ipython-input-5-1e0a8ce526a7>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  ChatOpenAI(model="gpt-4", temperature=0, openai_api_key= OPENAI_API_KEY),


### DuckDuckGo Logic

In [6]:
search = DuckDuckGoSearchResults()
class SummarizeText(BaseModel):
    """Model for text to be summarized."""
    text: str = Field(..., title="Text to summarize", description="The text to be summarized")

In [7]:
def parse_search_results(results_string: str) -> List[dict]:
    """Parse a string representation of search results into a list of dictionaries."""
    results = []
    entries = results_string.split(', snippet: ')
    for entry in entries[1:]:  # Skip the first split as it's empty
        parts = entry.split(', title: ')
        if len(parts) == 2:
            snippet = parts[0]
            title_link = parts[1].split(', link: ')
            if len(title_link) == 2:
                title, link = title_link
                results.append({
                    'snippet': snippet,
                    'title': title,
                    'link': link
                })
    return results

def perform_web_search(query: str, specific_site: Optional[str] = None) -> Tuple[List[str], List[Tuple[str, str]]]:
    """Perform a web search based on a query, optionally including a specific website."""
    try:
        if specific_site:
            specific_query = f"site:{specific_site} {query}"
            print(f"Searching for: {specific_query}")
            specific_results = search.run(specific_query)
            print(f"Specific search results: {specific_results}")
            specific_parsed = parse_search_results(specific_results)

            general_query = f"-site:{specific_site} {query}"
            print(f"Searching for: {general_query}")
            general_results = search.run(general_query)
            print(f"General search results: {general_results}")
            general_parsed = parse_search_results(general_results)

            combined_results = (specific_parsed + general_parsed)[:3]
        else:
            print(f"Searching for: {query}")
            web_results = search.run(query)
            print(f"Web results: {web_results}")
            combined_results = parse_search_results(web_results)[:3]

        web_knowledge = [result.get('snippet', '') for result in combined_results]
        sources = [(result.get('title', 'Untitled'), result.get('link', '')) for result in combined_results]

        print(f"Processed web_knowledge: {web_knowledge}")
        print(f"Processed sources: {sources}")
        return web_knowledge, sources
    except Exception as e:
        print(f"Error in perform_web_search: {str(e)}")
        import traceback
        traceback.print_exc()
        return [], []

def summarize_text(text: str, source: Tuple[str, str]) -> str:
    """Summarize the given text using OpenAI's language model."""
    try:
        # Instantiate your ChatOpenAI with a desired model and temperature.
        llm = ChatOpenAI(temperature=0.7, model="gpt-4o-mini",openai_api_key=OPENAI_API_KEY)
        prompt_template = "Please summarize the following text in a listed bullet point format:\n\n{text}\n\nSummary:"
        prompt = PromptTemplate(
            template=prompt_template,
            input_variables=["text"],
        )
        # Chain the prompt with the LLM (here we use a simple invocation)
        summary_chain = prompt | llm
        input_data = {"text": text}
        summary = summary_chain.invoke(input_data)

        # If the summary has a content attribute, use it; otherwise convert to string.
        summary_content = summary.content if hasattr(summary, 'content') else str(summary)
        formatted_summary = f"Source: {source[0]} ({source[1]})\n{summary_content.strip()}\n"
        return formatted_summary
    except Exception as e:
        print(f"Error in summarize_text: {str(e)}")
        return ""

def search_summarize(query: str, specific_site: Optional[str] = None) -> str:
    """Perform a web search and summarize the results."""
    web_knowledge, sources = perform_web_search(query, specific_site)

    if not web_knowledge or not sources:
        print("No web knowledge or sources found.")
        return ""

    # Create a list of summaries—only add non-empty summaries
    summaries = []
    for knowledge, source in zip(web_knowledge, sources):
        summary = summarize_text(knowledge, source)
        if summary:
            summaries.append(summary)

    combined_summary = "\n".join(summaries)
    return combined_summary

# -------------------------------------------------------------------
# Wrap the above `search_summarize` function as a LangChain Tool
# -------------------------------------------------------------------

duckduckgo_search_summarize_tool = Tool(
    name="DuckDuckGoSearchSummarize",
    func=search_summarize,
    description=(
        "Performs a web search using DuckDuckGo based on the input query "
        "and summarizes the top 3 search results using an OpenAI LLM. "
        "Optionally, you can restrict the search to a specific site by providing the 'specific_site' parameter."
    )
)




### Tools

In [8]:
# Calculator Tool (basic arithmetic)
def calculator_tool(expression: str) -> str:
    try:
        result = eval(expression)  # In production, consider a safer math parser.
        return f"The result of '{expression}' is {result}."
    except Exception as e:
        return f"Error evaluating expression '{expression}': {e}"

calculator = Tool(
    name="Calculator",
    func=calculator_tool,
    description="Evaluates simple mathematical expressions, e.g. '2+2'."
)

# Python Code Execution Tool
def code_execution_tool(code: str) -> str:
    """
    Executes Python code in a restricted environment.
    WARNING: Using exec can be a security risk. Use a sandbox in production.
    """
    import io
    import contextlib

    output_buffer = io.StringIO()
    try:
        with contextlib.redirect_stdout(output_buffer):
            exec(code, {"__builtins__": {}})
        return output_buffer.getvalue() or "Code executed successfully with no output."
    except Exception as e:
        return f"Error executing code: {str(e)}"

code_executor = Tool(
    name="CodeExecutor",
    func=code_execution_tool,
    description="Executes a snippet of Python code and returns its output. Use carefully!"
)

# Wikipedia Summary Tool
def wikipedia_tool(query: str) -> str:
    try:
        summary = wikipedia.summary(query, sentences=2)
        return summary
    except Exception as e:
        return f"Could not retrieve Wikipedia summary for '{query}': {str(e)}"

wikipedia_search = Tool(
    name="WikipediaSearch",
    func=wikipedia_tool,
    description="Looks up a summary of a topic using Wikipedia."
)

# Translation Tool (using deep-translator)
def translation_tool(text: str, target_language: str = "en") -> str:
    try:
        translated_text = GoogleTranslator(source='auto', target=target_language).translate(text)
        return f"Translated text: {translated_text}"
    except Exception as e:
        return f"Translation error: {str(e)}"

translator = Tool(
    name="Translator",
    func=translation_tool,
    description="Translates the given text into a specified language (default is English)."
)

# Sentiment Analysis Tool (simple keyword-based approach)
def sentiment_analysis_tool(text: str) -> str:
    lower_text = text.lower()
    if any(word in lower_text for word in ["happy", "excellent", "great", "good"]):
        return "Positive sentiment detected."
    elif any(word in lower_text for word in ["sad", "bad", "terrible", "awful"]):
        return "Negative sentiment detected."
    else:
        return "Neutral or no clear sentiment detected."

sentiment_analyzer = Tool(
    name="SentimentAnalysis",
    func=sentiment_analysis_tool,
    description="Analyzes the sentiment of a given text (positive, negative, or neutral)."
)


def pandas_analysis_tool(query: str) -> str:
    """
    This tool calls the Pandas DataFrame agent with a natural language query.
    Returns the agent's response as a string.
    """
    try:
        response = pandas_agent.run(query)
        return str(response)
    except Exception as e:
        return f"Error in Pandas DataFrame tool: {str(e)}"

pandas_tool = Tool(
    name="PandasDataAnalysis",
    func=pandas_analysis_tool,
    description=(
        "Use this tool to ask questions about the loaded Pandas DataFrame. "
        "For example, you can filter, describe statistics, or transform the data."
    )
)

duckduckgo_search_summarize_tool = Tool(
    name="DuckDuckGoSearchSummarize",
    func=search_summarize,
    description=(
        "Performs a web search using DuckDuckGo based on the input query "
        "and summarizes the top 3 search results using an OpenAI LLM. "
        "Optionally, you can restrict the search to a specific site by providing the 'specific_site' parameter."
    )
)

# --- 3. Combine Tools into a Single Agent -------------------------------------

tools = [
    calculator,
    code_executor,
    wikipedia_search,
    translator,
    sentiment_analyzer,
    pandas_tool,
    duckduckgo_search_summarize_tool
]

In [9]:
# Initialize the OpenAI language model with your API key.
# Replace 'your_openai_api_key_here' with your actual API key.
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=OPENAI_API_KEY)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Initialize the agent with the provided tool and language model.
agent = initialize_agent(
    tools=tools,
    llm=llm,
    allow_dangerous_code=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # ReAct agent
    verbose=True,  # see chain-of-thought reasoning
    memory=memory
)

<ipython-input-9-72d3c6149a04>:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
<ipython-input-9-72d3c6149a04>:8: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [10]:
query = (
        """
        1) Calculate 3*7.
        2) Check a brief Wikipedia summary about 'Large language models'.
        3) Translate the summary to Spanish.
        4) Analyze if the text is positive or negative.
        5) Summarize it in 1 paragraph.
        6) Then show me a simulated web search on that topic.
        """
        )

print("USER QUERY:", query)
result = agent.run(query)
print("\nAGENT RESULT:\n", result)

<ipython-input-10-5723d4c5ffed>:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run(query)


USER QUERY: 
        1) Calculate 3*7.
        2) Check a brief Wikipedia summary about 'Large language models'.
        3) Translate the summary to Spanish.
        4) Analyze if the text is positive or negative.
        5) Summarize it in 1 paragraph.
        6) Then show me a simulated web search on that topic.
        


> Entering new AgentExecutor chain...
The question is asking for multiple tasks. Let's break it down and handle each task one by one. The first task is a simple multiplication operation.
Action: Calculator
Action Input: '3*7'
Observation: The result of ''3*7'' is 3*7.
Thought:The multiplication operation was successful. Now, let's move on to the second task which is to get a brief Wikipedia summary about 'Large language models'.
Action: WikipediaSearch
Action Input: 'Large language models'
Observation: A large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. LLMs are language models

In [11]:


query = (
    """
    "Search online the benefits of renewable energy"
    """
)

print("USER QUERY:", query)
result = agent.run(query)
print("\nAGENT RESULT:\n", result)

USER QUERY: 
    "Search online the benefits of renewable energy"
    


> Entering new AgentExecutor chain...
I need to perform an online search to find information about the benefits of renewable energy.
    
Action: 
    DuckDuckGoSearchSummarize
    
Action Input: 
    "benefits of renewable energy" 
    Searching for: benefits of renewable energy" 

Web results: snippet: Some of the benefits of renewable energy hit headlines, others are well known. But some benefits aren't even primary benefits at all, and are happy by-products of other pursuits. So for this week's Top 10, we run through some of the benefits that renewable energy brings businesses, individuals, governments and countries, to name a few. 10., title: Top 10: Benefits of Renewable Energy | Energy Magazine - Energy Digital, link: https://energydigital.com/top10/top-10-benefits-of-renewable-energy, snippet: Purchasing renewable energy from an electric utility through a green pricing or green marketing program, where buy

In [12]:
query = (
    """
    Follow the ReAct framework and create a list of all the benefits that a Wells Fargo employee can earn

    """
)

print("USER QUERY:", query)
result = agent.run(query)
print("\nAGENT RESULT:\n", result)

USER QUERY: 
    Follow the ReAct framework and create a list of all the benefits that a Wells Fargo employee can earn

    


> Entering new AgentExecutor chain...
I need to find information about the benefits that Wells Fargo provides to its employees. This information is likely available on the Wells Fargo website or on websites that provide information about company benefits. 

Action: DuckDuckGoSearchSummarize
Action Input: Wells Fargo employee benefitsSearching for: Wells Fargo employee benefits
Web results: snippet: Wells Fargo employee benefits and perks, which include health insurance benefits, transport facilities, work from home policy, education assistance, etc. Reported by 3484 Wells Fargo employees ., title: Wells Fargo Employee Benefits in 2025 | Ambitionbox, link: https://www.ambitionbox.com/benefits/wells-fargo-benefits, snippet: Learn how to qualify for the Wells Fargo employee discount, explore available offers, and understand key enrollment steps and considerations.

## LlamaIndex

In [13]:
!pip install llama-index gradio -qqq
!pip install llama-index-readers-file
!pip install langchain openai
# !pip install --upgrade llama-index langchain openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-legacy 0.9.48.post4 requires requests>=2.31.0, but you have requests 2.29.0 which is incompatible.
llama-index-core 0.12.30 requires requests>=2.31.0, but you have requests 2.29.0 which is incompatible.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.29.0 which is incompatible.
sphinx 8.2.3 requires requests>=2.30.0, but you have requests 2.29.0 which is incompatible.
yfinance 0.2.55 requires requests>=2.31, but you have requests 2.29.0 which is incompatible.
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.29.0
    Uninstalling requests-2.29.0:
      Successfully uninstalled requests-2.29.0
ERROR: pip's dependency resolver does not cu

In [14]:
import os
import logging
import sys
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
    Document,
    GPTVectorStoreIndex,
    readers
)

from llama_index.readers.file import (
    PDFReader,
    EpubReader,
    FlatReader,
    HTMLTagReader,
    ImageCaptionReader,
    ImageReader,
    ImageVisionLLMReader,
    IPYNBReader,
    MarkdownReader,
    MboxReader,
    PptxReader,
    PandasCSVReader,
    VideoAudioReader,
    UnstructuredReader,
    PyMuPDFReader,
    ImageTabularChartReader,
    XMLReader,
    PagedCSVReader,
    CSVReader,
    RTFReader,
)
import gradio as gr

ImportError: cannot import name 'VideoAudioParser' from 'llama_index.readers.file.video_audio' (/usr/local/lib/python3.11/dist-packages/llama_index/readers/file/video_audio/__init__.py)

In [ ]:
# Create some example documents.
documents = SimpleDirectoryReader("/content/TestingDocs").load_data()

# Build your index over the documents. This index will allow the agent to retrieve relevant document snippets.
index = GPTVectorStoreIndex(documents)

In [ ]:
# Initialize a ChatOpenAI instance; configure parameters like temperature as needed.
llm = ChatOpenAI(temperature=0.7)

# Create a conversation memory instance that will store the dialogue history.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


def process_query(query: str):
    # Retrieve the previous conversation history from memory.
    history = memory.load_memory_variables({})
    # Format history into a string.
    conversation_context = ""
    for msg in history.get("chat_history", []):
        conversation_context += f"{msg['role'].capitalize()}: {msg['content']}\n"

    # Query the index for information relevant to the current query.
    index_response = index.query(query)

    # Combine previous conversation, the current query, and retrieved index information into a prompt.
    prompt = (
        f"{conversation_context}"
        f"User: {query}\n"
        f"Index information: {index_response}\n"
        f"Assistant:"
    )

    # Get the LLM to produce an answer based on the combined prompt.
    final_response = llm.predict(prompt)

    # Output the agent's response.
    print("Agent:", final_response)

    # Save the current interaction to the conversation memory for future context.
    memory.save_context({"input": query}, {"output": final_response})


# Run an Interactive Loop
# ------------------------------
print("Welcome! Type your queries (or 'exit' to quit):")
while True:
    user_input = input("Enter your query: ")
    if user_input.lower().strip() == "exit":
        break
    process_query(user_input)


## Tree Of Thought Simple Example

In [17]:
#!/usr/bin/env python
import os
from typing import List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.tools import Tool

# -------------------------------
# Reasoner Agent: Generate Multiple Reasoning Paths
# -------------------------------
def generate_reasoning_paths(query: str, num_paths: int = 3) -> List[str]:
    """
    Generates multiple distinct reasoning paths for a given query.
    Uses a Tree-of-Thought style prompt to have the LLM propose different paths.
    """
    llm = ChatOpenAI(temperature=0.7, model="gpt-4", openai_api_key=os.getenv("OPENAI_API_KEY"))
    prompt_template = PromptTemplate(
        template=(
            "You are a reasoning agent. Given the following query: '{query}', "
            "generate {num_paths} distinct reasoning paths. "
            "For each path, include detailed chain-of-thought steps and a final answer. "
            "Format your response as follows:\n\n"
            "Path 1:\n<reasoning steps>\nAnswer: <final answer>\n\n"
            "Path 2:\n<reasoning steps>\nAnswer: <final answer>\n\n"
            "Path 3:\n<reasoning steps>\nAnswer: <final answer>\n\n"
            "Ensure each path is logically sound."
        ),
        input_variables=["query", "num_paths"]
    )
    prompt = prompt_template.format(query=query, num_paths=num_paths)
    response = llm.predict(prompt)
    # Assume the response is structured with "Path 1:" markers
    paths = []
    for segment in response.split("Path "):
        segment = segment.strip()
        if segment:
            # Prepend "Path " back to each segment for clarity
            paths.append("Path " + segment)
    return paths[:num_paths]

# -------------------------------
# Thought Validator Agent: Validate Reasoning
# -------------------------------
def validate_reasoning_path(reasoning: str) -> bool:
    """
    Validates a reasoning path using a dedicated prompt.
    Returns True if the reasoning is valid, or False otherwise.
    """
    llm = ChatOpenAI(temperature=0.5, model="gpt-4", openai_api_key=os.getenv("OPENAI_API_KEY"))
    prompt_template = PromptTemplate(
        template=(
            "You are a Thought Validator. Evaluate the following reasoning chain "
            "and determine if it is logically sound, factually accurate, and complete.\n\n"
            "Reasoning:\n{reasoning}\n\n"
            "Respond with only 'Valid' if the reasoning is acceptable, or 'Invalid' if it is not, "
            "followed by a brief explanation."
        ),
        input_variables=["reasoning"]
    )
    prompt = prompt_template.format(reasoning=reasoning)
    validation_response = llm.predict(prompt)
    # If response starts with "Valid" (ignore case), then consider it valid
    if validation_response.strip().lower().startswith("valid"):
        return True
    else:
        return False

# -------------------------------
# Combined Multi-Agent ToT Reasoner with Thought Validator
# -------------------------------
def multi_agent_tot_reasoner(query: str) -> str:
    """
    Orchestrates the multi-agent Tree-of-Thought reasoning process:
      1. Generates multiple reasoning paths for the query.
      2. Validates each path.
      3. Selects a consensus (first valid) path.
      4. Returns a detailed output showing all reasoning paths along with their validation status,
         the chosen consensus path, and the final answer extracted from that path.
    """
    # Generate multiple reasoning paths
    paths = generate_reasoning_paths(query, num_paths=3)

    # Validate each path and record its status (True=Valid, False=Invalid)
    valid_flags = []
    for path in paths:
        is_valid = validate_reasoning_path(path)
        valid_flags.append(is_valid)

    # Select the first valid path; if none are valid, choose the first path as a fallback
    consensus_idx = None
    for i, valid in enumerate(valid_flags):
        if valid:
            consensus_idx = i
            break
    if consensus_idx is None:
        consensus_idx = 0  # Fallback if no path is valid

    consensus_path = paths[consensus_idx]

    # Extract the final answer from the consensus path
    # Look for a line that starts with "Answer:" (case-insensitive)
    final_answer = "No final answer found."
    for line in consensus_path.splitlines():
        if line.strip().lower().startswith("answer:"):
            final_answer = line.split("Answer:", 1)[1].strip()
            break

    # Build a string that includes all paths and marks each as VALID/INVALID
    all_paths_str = ""
    for i, (path, is_valid) in enumerate(zip(paths, valid_flags), start=1):
        status = "VALID" if is_valid else "INVALID"
        all_paths_str += f"--- Path {i} ({status}) ---\n{path}\n\n"

    result = (
        f"All Generated Reasoning Paths:\n{all_paths_str}"
        f"--- Consensus Path (Path {consensus_idx+1}) ---\n{consensus_path}\n\n"
        f"Final Answer: {final_answer}"
    )
    return result

# -------------------------------
# Wrap as a LangChain Tool
# -------------------------------
tot_reasoner_tool = Tool(
    name="MultiAgentToTReasoner",
    func=multi_agent_tot_reasoner,
    description=(
        "A multi-agent Tree-of-Thought reasoning tool with a Thought Validator. "
        "Generates multiple reasoning paths for a given query, validates them, "
        "and returns all paths (with their VALID/INVALID status), along with a consensus reasoning "
        "and the final answer extracted from it."
    )
)

In [18]:
# -------------------------------
# Example Integration into a LangChain Agent
# -------------------------------
if __name__ == "__main__":
    # Set your API key here or use your environment configuration
    os.environ["OPENAI_API_KEY"] = ""

    # Example query for complex reasoning
    query = "How can I optimize a sorting algorithm in Python for large datasets?"
    answer = tot_reasoner_tool.func(query)
    print("Multi-Agent ToT Reasoner Tool Output:\n", answer)

Multi-Agent ToT Reasoner Tool Output:
 All Generated Reasoning Paths:
--- Path 1 (VALID) ---
Path 1:
1. The first step is understanding the need for optimization. Large datasets can lead to inefficient sorting if not properly optimized.
2. The next step is to identify the current sorting algorithm used. Different algorithms have different strengths and weaknesses and the optimal one often depends on the nature of the data.
3. Python's built-in sort function uses a method called Timsort, which is highly efficient for most datasets. However, its efficiency can be improved with certain optimization techniques.
4. One technique is to reduce function call overhead by using in-place sorting. This means the list is sorted without creating a new list, minimizing memory usage and improving speed.
5. Another technique is to use a binary insertion sort, a variant of the traditional insertion sort that uses binary search to find the correct location for insertion. This can significantly reduce the